In https://www.bogleheads.org/forum/viewtopic.php?f=10&t=192105&start=200#p2986911
longinvest said, "It would be quite interesting to see the portfolio and withdrawal
trajectory of 30/70, 40/60, and maybe 50/50 (stocks/bonds) constant AA portfolios vs
Prime Harvesting in both nominal and inflation-adjusted term, and look at the result
with the eye of a retiree who sees nominal numbers on his bank account and mutual fund
statements.

So let's try to do that.

[siamond has an investigation here](https://www.bogleheads.org/forum/viewtopic.php?f=10&t=192105&start=200#p2986703)
My charts and his look the same, which is good. (Though he shows 40 years and I only show 25)

He implements VPW a bit differently than I do, so the VPW charts look different.
- He doesn't take into account the changing asset allocation
- He uses a longer-than-default timespan (50 years instead of 35)

How to judge the success or failure of Prime Harvesting?

The most important thing is how much money you get over your life time. If you don't get more money it isn't
even worth looking at other parameters.

Only *after* that should we start looking at measures of risk. What kinds of measures of risk?
- Total portfolio balance
- Standard deviation
- Percentage of bonds
- Milevsky's "risk quotient"?

Use stochastic mortality.

In [1]:
%matplotlib inline
from decimal import Decimal
import itertools
from pprint import pprint
import math

import pandas
import seaborn
from matplotlib import pyplot as plt
plt.style.use('seaborn-poster')

import metrics
import simulate
import harvesting
import market
import plot
import mortality
from plot import plt
import withdrawal
from portfolio import Portfolio
import montecarlo

%pdb on

Automatic pdb calling has been turned ON


In [95]:
#series = market.Returns_US_1871()
#START_YEAR=1871

#series = market.Japan_1957()
#START_YEAR=1957

#series = market.UK1900()
#START_YEAR = 1900

series = market.JST('PRT')
START_YEAR = series.start_year

In [96]:
x = simulate.withdrawals(series.iter_from(1929),
                                 #withdraw=lambda p, s: withdrawal.LonginvestSmoothing(withdrawal.VPW(p, s, years_left=50)),
                                 #withdraw=lambda p, s: withdrawal.ConstantDollar(p, s, rate=Decimal('.04')),
                                 withdraw=withdrawal.Clyatt,
                                 years=30,
                                 portfolio=(600000, 400000),
                                 harvesting=harvesting.make_rebalancer(.6))
[float(n.withdraw_n) for n in x]

[50000.0,
 47584.98,
 45205.73,
 42945.44,
 40798.17,
 39015.29,
 47277.2,
 47854.24,
 49456.69,
 48443.17,
 48043.92,
 45641.72,
 43359.64,
 46215.42,
 66731.21,
 65788.22,
 67636.34,
 68668.84,
 65235.4,
 61973.63,
 58874.95,
 55931.2,
 53134.64,
 50477.91,
 47954.01,
 46522.52,
 54454.25,
 51731.53,
 49240.48,
 46778.45]

In [97]:
def test_all(year, strategies_to_test, years=25):
    results = {}
    for h in strategies_to_test:
        starting_portfolio = (600000,400000)

        # siamond used VPW with a 50 year depletion period, so I'll try that as well
        x = simulate.withdrawals(series.iter_from(year),
                                 withdraw=lambda p, s: withdrawal.VPW(p, s, years_left=40),
#                                 withdraw=withdrawal.Clyatt,
#                                 withdraw=lambda p, s: withdrawal.ConstantDollar(p, s, rate=Decimal('.04')),
#                                 withdraw=lambda p, s: withdrawal.SteinerSmoothing(withdrawal.VPW(p, s, years_left=40)),
#                                 withdraw=lambda p, s: withdrawal.ECM(p, s),
#                                 withdraw=lambda p, s: withdrawal.Model3(p, s, start_rate=0.01, floor_rate=0.01),
                                 years=years,
                                 portfolio=starting_portfolio,
                                 harvesting=h)
        results[h.__name__] = x
    return results

In [98]:
def make_data_tables(strategies_to_test, lens, years=25):
    frames = {}
    
    for s in strategies_to_test:
        frames[s.__name__] = pandas.DataFrame(columns=['Y%02d' % y for y in range(years)])
        
    last_year = 2015 - years
    
    for i in range(START_YEAR, last_year):
        n = test_all(i, strategies_to_test, years=years)
        for s in n.keys():
            frames[s].loc[i] = [lens(_) for _ in n[s]]

    return frames

Kurtosis is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution. That is, data sets with high kurtosis tend to have heavy tails, or outliers. Data sets with low kurtosis tend to have light tails, or lack of outliers. A uniform distribution would be the extreme case.

We prefer lower kurtosis.

The skewness for a normal distribution is zero, and any symmetric data should have a skewness near zero. Negative values for the skewness indicate data that are skewed left and positive values for the skewness indicate data that are skewed right. By skewed left, we mean that the left tail is long relative to the right tail. Similarly, skewed right means that the right tail is long relative to the left tail. If the data are multi-modal, then this may affect the sign of the skewness.

We want skewness to be higher.

In [99]:
def semideviation(frame):
    #goal = frame.mean()
    goal = 40000
    values = frame[lambda s: s < goal]
    sumvalues = sum(((goal - v) ** 2 for v in values))
    average = sumvalues / len(values)
    return math.sqrt(average)


def calculate_stuff(df, use_all_columns=True):
    """
    Things we don't yet calculate.
    Real returns (that an investor sees)
    Average asset allocation of bonds
    Minimum asset allocation of bond
    """
    
    if use_all_columns:
        stack = df.stack()
    else:
        columns = df.columns.tolist()
        stack = df[columns[-1]]
    
    return ({
            'Mean': round(stack.mean()),
            'Median' : round(stack.median()),
            'Stddev': stack.std() / stack.mean(),
            'Min': stack.min(),
            'Max' : stack.max(),
            '0.1h percentile' : round(stack.quantile(.001)),        
            ' 1st percentile' : round(stack.quantile(.01)),
            ' 5th percentile' : round(stack.quantile(.05)),
            '10th percentile' : round(stack.quantile(.1)),
            '90th percentile' : round(stack.quantile(.9)),
#            'Mean of 25% lowest' : round(stack.nsmallest(int(len(stack) / 4)).mean()),
            'Kurtosis' : stack.kurtosis(),
            'Skew' : stack.skew(),
            'Semidev-4' : semideviation(stack),
    })

In [100]:
def make_all_stats():
    strategies_to_test = [
#        harvesting.N_35_RebalanceHarvesting,
        harvesting.N_60_RebalanceHarvesting,
#        harvesting.make_rebalancer(0.8),
#        harvesting.N_100_RebalanceHarvesting,
#        harvesting.BondsFirst,
#        harvesting.WoodSpinner,
#        harvesting.AltPrimeHarvesting,
        harvesting.PrimeHarvesting,
#        harvesting.AgeBased_100,
#        harvesting.AgeBased_110,
#        harvesting.AgeBased_120,
#        harvesting.Glidepath,
#        harvesting.OmegaNot,
#        harvesting.Weiss,
#        harvesting.ActuarialHarvesting,
#        harvesting.InverseGlidepath,
    ]

    #t = make_data_tables(strategies_to_test, lambda x: round(x.portfolio_r), years=40)
    t = make_data_tables(strategies_to_test, lambda x: round(x.withdraw_r), years=30)
    #t = make_data_tables(strategies_to_test, lambda x: round(x.portfolio_bonds/x.portfolio_n*100), years=30)

    if False:
        fn_mort = mortality.make_mortality_rate()
        for key in t:
            age = 65
            for c in t[key].columns:
                t[key][c] *= (1 - fn_mort(age, mortality.FEMALE))
                age += 1


    if False:
        for k in t:
            t[k].to_csv('CSV - withdraw - %s.csv' % k)
            

    df = None

    for key in sorted(t.keys()):
        #t[key].to_csv('CSV - portfolio - %s.csv' % key)
        stats = calculate_stuff(t[key], use_all_columns=True)

        if df is None:
            # We need to know the columns in order to define a data frame,
            # so we defer the creation until now
            df = pandas.DataFrame(columns=sorted(list(stats.keys())))
        df.loc[key] = stats
        
        #seaborn.distplot(t[key].stack(), label=key, axlabel='Annual Withdrawal ($$$)', color=['red', 'blue'])
        #plt.legend(loc=0)

    return df

d = make_all_stats()
#d.to_csv('CSV-comparison.csv')
d

,1st percentile,5th percentile,0.1h percentile,10th percentile,90th percentile,Kurtosis,Max,Mean,Median,Min,Semidev-4,Skew,Stddev
AnnualRebalancer_60,20301.0,34519.0,11806.0,41577.0,212161.0,22.683292,1319748.0,109443.0,69682.0,8842.0,11928.809896,4.213637,1.152656
PrimeHarvesting,25314.0,39052.0,11746.0,45598.0,242342.0,16.676116,1123937.0,115846.0,71353.0,9569.0,11231.492523,3.639043,1.101977


In [ ]:
def compare_year_lens(year, lens, title):
    strategies_to_test = [
        harvesting.N_60_RebalanceHarvesting,
#        harvesting.N_100_RebalanceHarvesting,
#        harvesting.N_0_RebalanceHarvesting,
#        harvesting.N_35_RebalanceHarvesting,
#        harvesting.OmegaNot,
#        harvesting.Weiss,
#        harvesting.BondsFirst,
#        harvesting.make_rebalancer(0.8),
        harvesting.WoodSpinner,        
#        harvesting.AltPrimeHarvesting,
#        harvesting.PrimeHarvesting,
#        harvesting.InverseGlidepath,
#        harvesting.ActuarialHarvesting,
    ]

    results = test_all(year, strategies_to_test, years=30)
    
    fig, ax = plt.subplots()

    if '%' not in title:
        plot.format_axis_labels_with_commas(ax.get_yaxis())

    plt.xlabel('Year of Retirement')
    plt.title('Retiring in %s (%s)' % (year, title))

    for strategy in (sorted(results.keys())):
        ax_n = fig.add_subplot(111, sharex=ax, sharey=ax)
        ws = [lens(n) for n in results[strategy]]
        ax_n.plot(ws, label=strategy)
        ax_n.set_ymargin(0.05)
        
        smallest = min(ws)
        index_of_smallest = ws.index(smallest)
        print(smallest)
#        ax_n.annotate("${:,}".format(int(smallest)), xy=(index_of_smallest, smallest),
#             xytext=(index_of_smallest - 3, smallest - 15000),
#             arrowprops=dict(facecolor='black', connectionstyle="arc3,rad=.2"),
#             fontsize=14)

    plt.legend(loc=0)
    ax.set_ylim(bottom=0)
        
    plt.show()
    
    s = {}
    for strategy in results.keys():
        ws = [lens(n) for n in results[strategy]]
        s[strategy] = ws
    df = pandas.DataFrame(data=s)
#    diff = (df['60% Stocks'] - df['100% Stocks'])
#    print(diff.sum())
#    print(diff.loc[lambda x: x > 0].mean())
#    print(diff)

In [ ]:
def chart_all(year):
#    compare_year_lens(year, lambda x: x.portfolio_stocks/x.portfolio_n*100, "Stock %")
#    compare_year_lens(year, lambda x: x.portfolio_n, "Portfolio (Nominal)")
#    compare_year_lens(year, lambda x: x.portfolio_r, "Portfolio (Real)")
#     compare_year_lens(year, lambda x: x.withdraw_n, "Withdrawals (Nominal)")
    compare_year_lens(year, lambda x: x.withdraw_r, "Withdrawals (Real)")
    
chart_all(1950)